In [1]:
from keras.preprocessing.sequence import pad_sequences

/usr/lib/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/lib/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import re
import numpy as np

In [3]:
inputs = ["what is <> doing?","what is being done by <>?","<> is upto what?"]

In [4]:
names = ["john","aman","aparna","dog","amit","ketan"]
actions = ["dancing","playing","singing","skating","riding","climbing"]

In [5]:
def sample_utter(n,act_name):
    opts = [
        n+' is '+act_name,
        act_name,
        "I think "+n+' is '+act_name,
        "I guess"+n+' is '+act_name,
        n+' is '+act_name+" I believe.",
        n+' is '+act_name+" is my guess",
    ]
    return opts[np.random.randint(len(opts))]

In [6]:
X=[]
X_name = []
Y=[]
N=[]
def gen_sent(inputs,names,actions):
    for n in names:
        for ii in range(len(inputs)*100):
            i = inputs[ii%len(inputs)]
            X.append(re.sub(r'<>',n,i))
            X_name.append(n)
            num=np.random.randint(len(actions)*100)
            Y.append(sample_utter(n,actions[num%len(actions)]))
            N.append(num)

In [33]:
Y[:5]

['john is riding is my guess',
 'john is skating',
 'john is playing',
 'I guessjohn is skating',
 'skating']

In [7]:
gen_sent(inputs,names,actions)

In [11]:
import keras
from keras.preprocessing.text import Tokenizer,one_hot
from keras.models import Model
from keras.layers import Input,Dense
from keras.utils import to_categorical

In [12]:
t=Tokenizer()

In [13]:
t.fit_on_texts(X)
t.fit_on_texts(Y)

In [14]:
t.index_word = {i:w for w,i in t.word_index.items()}

In [15]:
vocab_len=len(t.word_index.keys())+1

# DB_model

In [16]:
# modelling the db
m_in = Input(shape=(vocab_len,))
m = Dense(100,activation='relu')(m_in)
m = Dense(vocab_len,activation='sigmoid')(m)
db_model = Model(inputs=[m_in],outputs=[m])
db_model.compile(loss='binary_crossentropy',optimizer='adam')

In [17]:
def softmax(w, t = 1.0):
    e = np.exp(np.array(w) / t)
    dist = e / np.sum(e)
    return dist

In [18]:
np.max(softmax(np.random.normal(size=vocab_len,loc=100,scale=5)))

0.9835306461587442

In [19]:
db_X = []
db_Y = []
for _ in range(1000):
    vec = softmax(np.random.normal(size=vocab_len,loc=100,scale=5))
    v= t.index_word[np.argmax(vec)] if np.max(vec)>0.5 and np.argmax(vec)!=0 else ''
    for _ in range(100):
        db_X.append(vec)
        if v not in names:
            db_Y.append(np.array([0  for ii in range(vocab_len)]))
            continue
        num=np.random.randint(len(actions))
        ix = t.word_index[actions[num]]
        db_Y.append(np.array(to_categorical([ix],num_classes=vocab_len))[0])
dist=np.random.permutation(len(db_X))
db_Y = np.array(db_Y)[dist]
db_X = np.array(db_X)[dist]

In [20]:
def db_model_fit(epochs):
    db_model.fit(np.array(db_X),np.array(db_Y),epochs=epochs,validation_split=0.2,batch_size=64)

In [21]:
db_model_fit(2)

Train on 80000 samples, validate on 20000 samples
Epoch 1/2
80000/80000 [==============================] - 6s 80us/step - loss: 0.0577 - val_loss: 0.0169
Epoch 2/2
80000/80000 [==============================] - 5s 58us/step - loss: 0.0164 - val_loss: 0.0162


## testing db_model

In [89]:
to_categorical(t.texts_to_sequences(["Aman what doing"]),num_classes=vocab_len)

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],
      dtype=float32)

In [90]:
y=db_model.predict(to_categorical(t.texts_to_sequences(["Aman what doing"]),num_classes=vocab_len)[0])

In [91]:
y.argsort(axis=-1)

array([[ 0,  2,  5, 10, 24, 26,  4, 29, 16,  1, 30, 28,  9, 15,  3, 13,
         8, 12,  6, 11,  7, 23, 27, 14, 25, 19, 18, 21, 22, 20, 17],
       [ 4,  8, 13,  0,  7, 29, 16, 11, 15,  1, 28,  5, 26,  2,  6, 10,
         3, 25, 27,  9, 30, 24, 23, 12, 14, 19, 17, 21, 18, 20, 22],
       [13,  4,  8, 28,  1,  5, 25, 29, 26,  0, 10,  7, 30, 15, 16,  3,
         6, 11, 24,  9, 27,  2, 12, 14, 23, 21, 19, 18, 17, 22, 20]])

In [92]:
np.argsort(np.sum(db_Y,axis=0))

array([ 0, 28, 27, 26, 25, 24, 23, 16, 29, 14, 13, 12, 15, 10,  1,  2,  3,
        4,  5,  6, 11, 30,  7,  8,  9, 18, 22, 17, 19, 21, 20])

In [93]:
[[(t.index_word[i],_j[i]) if i!=0 else ('null',_j[i]) for i in j] for j,_j in zip(y.argsort(axis=-1),y) ]

[[('null', 0.00020549457),
  ('what', 0.00022035681),
  ('being', 0.00025001532),
  ('aparna', 0.00027015933),
  ('guess', 0.00027812005),
  ('guessamit', 0.0002902962),
  ('doing', 0.0003047225),
  ('guessdog', 0.00030590312),
  ('believe', 0.0003135274),
  ('is', 0.0003214629),
  ('guessaparna', 0.00032192966),
  ('guessketan', 0.00032545443),
  ('dog', 0.00032789205),
  ('think', 0.00034241384),
  ('i', 0.0003477852),
  ('aman', 0.00035251956),
  ('upto', 0.0003548911),
  ('amit', 0.0003607396),
  ('done', 0.00036165604),
  ('john', 0.00037062744),
  ('by', 0.00037775622),
  ('my', 0.00040202026),
  ('guessjohn', 0.00040648843),
  ('ketan', 0.00043073343),
  ('guessaman', 0.0004505105),
  ('climbing', 0.1547346),
  ('skating', 0.18767904),
  ('riding', 0.18811932),
  ('dancing', 0.18911086),
  ('singing', 0.19436792),
  ('playing', 0.20278767)],
 [('doing', 1.693764e-05),
  ('upto', 1.9104762e-05),
  ('aman', 1.9766618e-05),
  ('null', 2.0760584e-05),
  ('by', 2.3600442e-05),
  ('gu

In [23]:
y=db_model.predict(np.array([[0,0,0,0.9]+[0]*16]))

In [24]:
[[(t.index_word[i],_j[i]) if i!=0 else ('null',_j[i]) for i in j] for j,_j in zip(y.argsort(axis=-1),y) ]

[[('what', 0.00026142027),
  ('amit', 0.00028668376),
  ('dog', 0.00030894805),
  ('aman', 0.00031707078),
  ('done', 0.00032131217),
  ('by', 0.0003309926),
  ('is', 0.00035546705),
  ('john', 0.00036274322),
  ('ketan', 0.00036345888),
  ('being', 0.0003705994),
  ('doing', 0.00040941368),
  ('upto', 0.0004402966),
  ('aparna', 0.0004516661),
  ('null', 0.00045891947),
  ('skating', 0.17328088),
  ('climbing', 0.17810948),
  ('dancing', 0.18361813),
  ('riding', 0.18771142),
  ('playing', 0.18953821),
  ('singing', 0.19822973)]]

# Models

In [22]:
from keras.layers import LSTM
from keras.engine.network import Network

In [23]:
frozen_db_model = Network(
    m_in,
    m,
    name='frozen_db_model'
)
frozen_db_model.trainable=False

In [24]:
inp = Input((10,vocab_len))
out = LSTM(50)(inp)
out = Dense(vocab_len,activation='softmax')(out)
model_rnn=Model(inputs=[inp],outputs=out)

In [97]:

out = frozen_db_model(model_rnn.output)
model = Model(inputs=[model_rnn.input],outputs=out)

In [98]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [42]:
X_in=to_categorical(pad_sequences(t.texts_to_sequences(X),maxlen=10),num_classes=vocab_len)

In [39]:
Y_in = to_categorical([t.word_index[actions[n%len(actions)]] for n in N],num_classes=vocab_len)

# Hypothesis
For model_rnn we feed in sentences, it should output the names. And the names when queried against the database should give the correct result. 

To train the model_rnn, we use db_model and plug it in to get the final output. The idea is that db_model models the probability of the db to return the corresponding result given the output of model_rnn.

Hypothesis1: However since it only returns the probability and not the exact values, the training won't be possible. AS the model wouldn't know ever which value to predict for sure. And the association of the model_rnn output with the required final output wouldn't be complete.


Hypothesis2: If we train the db_model with the event id, such that for particular event id whatever the output was, it remembers with high accuracy. Then while training, event id could be an input for corresponding dataset. 

In [35]:
ii=866
X[ii:ii+1]


In [37]:
dist = np.random.permutation(len(X_in))
X_inr = X_in[dist]
Y_inr = Y_in[dist]

In [38]:
#model.fit(X_inr,Y_inr,epochs=20,validation_split=0.2)

Train on 1440 samples, validate on 360 samples
Epoch 1/20
1440/1440 [==============================] - 2s 1ms/step - loss: 0.4789 - acc: 0.9500 - val_loss: 0.4608 - val_acc: 0.9500
Epoch 2/20
1440/1440 [==============================] - 1s 734us/step - loss: 0.4575 - acc: 0.9500 - val_loss: 0.4556 - val_acc: 0.9500
Epoch 3/20
1440/1440 [==============================] - 1s 748us/step - loss: 0.4535 - acc: 0.9500 - val_loss: 0.4504 - val_acc: 0.9500
Epoch 4/20
1440/1440 [==============================] - 1s 752us/step - loss: 0.4480 - acc: 0.9500 - val_loss: 0.4465 - val_acc: 0.9500
Epoch 5/20
1440/1440 [==============================] - 1s 799us/step - loss: 0.4467 - acc: 0.9500 - val_loss: 0.4462 - val_acc: 0.9500
Epoch 6/20
1440/1440 [==============================] - 1s 770us/step - loss: 0.4467 - acc: 0.9500 - val_loss: 0.4462 - val_acc: 0.9500
Epoch 7/20
1440/1440 [==============================] - 1s 768us/step - loss: 0.4466 - acc: 0.9500 - val_loss: 0.4461 - val_acc: 0.9500
Epo

### testing model rnn

In [54]:
ii=1222
X[ii:ii+1]

['what is being done by amit?']

In [57]:
y=model_rnn.predict(X_in[ii:ii+1])
model.predict(X_in[ii:ii+1])

[array([[1.25020798e-02, 3.68301034e-01, 5.46059513e-04, 1.66156344e-04,
         7.81369527e-05, 3.28235074e-05, 4.48085302e-05, 3.89642737e-05,
         4.80694143e-05, 6.71076775e-02, 3.56861092e-02, 3.08005866e-02,
         1.11210831e-01, 1.25071099e-02, 3.58961403e-01, 1.03968167e-04,
         1.27062449e-04, 8.68929346e-05, 2.02584779e-04, 2.89392425e-04,
         9.37774093e-05, 2.70387769e-04, 7.99273912e-05, 6.83660401e-05,
         1.07351771e-04, 4.53218963e-05, 4.72883075e-05, 2.08147540e-04,
         5.35969339e-05, 6.72595415e-05, 1.16775205e-04]], dtype=float32),
 array([[7.68936734e-05, 9.30753522e-05, 8.32222722e-05, 9.60318794e-05,
         8.77991333e-05, 1.16250994e-04, 7.89057958e-05, 8.03349903e-05,
         9.11206516e-05, 8.43610323e-05, 1.08341628e-04, 7.03377300e-05,
         8.44032766e-05, 1.22309692e-04, 1.03045117e-04, 1.69003740e-01,
         1.40482083e-01, 9.70071123e-05, 1.61266521e-01, 9.93263820e-05,
         9.66861044e-05, 1.50057688e-01, 1.332525

In [56]:
y

array([[1.25020798e-02, 3.68301034e-01, 5.46059513e-04, 1.66156344e-04,
        7.81369527e-05, 3.28235074e-05, 4.48085302e-05, 3.89642737e-05,
        4.80694143e-05, 6.71076775e-02, 3.56861092e-02, 3.08005866e-02,
        1.11210831e-01, 1.25071099e-02, 3.58961403e-01, 1.03968167e-04,
        1.27062449e-04, 8.68929346e-05, 2.02584779e-04, 2.89392425e-04,
        9.37774093e-05, 2.70387769e-04, 7.99273912e-05, 6.83660401e-05,
        1.07351771e-04, 4.53218963e-05, 4.72883075e-05, 2.08147540e-04,
        5.35969339e-05, 6.72595415e-05, 1.16775205e-04]], dtype=float32)

In [138]:
t.index_word

{1: 'is',
 2: 'what',
 3: 'i',
 4: 'doing',
 5: 'being',
 6: 'done',
 7: 'by',
 8: 'upto',
 9: 'dog',
 10: 'aparna',
 11: 'john',
 12: 'amit',
 13: 'aman',
 14: 'ketan',
 15: 'think',
 16: 'believe',
 17: 'playing',
 18: 'skating',
 19: 'climbing',
 20: 'singing',
 21: 'riding',
 22: 'dancing',
 23: 'my',
 24: 'guess',
 25: 'guessaman',
 26: 'guessamit',
 27: 'guessjohn',
 28: 'guessketan',
 29: 'guessdog',
 30: 'guessaparna'}

In [139]:
db_model.predict(y)

array([[7.0457224e-07, 1.1879730e-06, 1.0322865e-06, 1.2272013e-06,
        7.3836287e-07, 6.8454915e-07, 1.5342617e-06, 1.1223832e-06,
        6.7522473e-07, 1.5305002e-06, 1.0025216e-06, 7.4134601e-07,
        1.0227470e-06, 7.7326280e-07, 1.3494027e-06, 1.0170628e-06,
        9.9952877e-07, 5.6951995e-07, 4.1391286e-07, 6.3639453e-07,
        3.8267402e-07, 5.9659351e-07, 5.7074391e-07, 1.3664762e-06,
        1.3437266e-06, 9.0083932e-07, 1.2790896e-06, 1.3683775e-06,
        1.3068997e-06, 9.4555094e-07, 1.2350961e-06]], dtype=float32)

In [140]:
[(t.index_word[i],y[0][i]) if i!=0 else '' for i in y.argsort(axis=-1)[0][-1::-1]]

[('is', 0.9997547),
 ('what', 0.00024442154),
 '',
 ('amit', 7.9605165e-08),
 ('dog', 6.358188e-08),
 ('john', 4.825296e-08),
 ('ketan', 3.139922e-08),
 ('aman', 2.9350415e-08),
 ('aparna', 2.6653815e-08),
 ('doing', 1.5053446e-08),
 ('done', 1.4722667e-08),
 ('upto', 1.279906e-08),
 ('being', 1.1407889e-08),
 ('guessaman', 9.183436e-09),
 ('by', 7.98632e-09),
 ('guessjohn', 7.702711e-09),
 ('guessketan', 7.2927953e-09),
 ('singing', 6.979357e-09),
 ('guess', 6.9270514e-09),
 ('guessdog', 6.1020584e-09),
 ('guessamit', 5.9828813e-09),
 ('my', 5.959431e-09),
 ('riding', 5.947905e-09),
 ('guessaparna', 5.6761262e-09),
 ('skating', 5.425327e-09),
 ('think', 5.19223e-09),
 ('dancing', 4.9146207e-09),
 ('believe', 3.632069e-09),
 ('climbing', 3.5582064e-09),
 ('i', 2.7873015e-09),
 ('playing', 2.6991402e-09)]

In [130]:
Y_in[0]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [117]:
[to_categorical(t.word_index['skating'],num_classes=vocab_len)]

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32)]

In [118]:
db_model.predict(np.array([to_categorical(t.word_index['skating'],num_classes=vocab_len)]))

array([[1.9607539e-06, 2.3289008e-06, 3.0498397e-06, 2.5500003e-06,
        1.5545529e-06, 2.0252571e-06, 3.7756481e-06, 2.2511813e-06,
        1.6099882e-06, 3.5145358e-06, 2.6573953e-06, 2.8243073e-06,
        3.2971404e-06, 2.0790701e-06, 3.3225817e-06, 1.8950619e-06,
        2.3109028e-06, 4.1045319e-06, 6.2974168e-06, 6.3568150e-06,
        6.2556492e-06, 5.8926685e-06, 3.6681006e-06, 3.0840336e-06,
        3.2481753e-06, 2.6144455e-06, 1.9214585e-06, 3.1966460e-06,
        2.8269239e-06, 1.8827100e-06, 3.0188132e-06]], dtype=float32)

## This validates our hypothesis as mode collapse happens quickly, and there is no gradient flow. 
Hypothesis 1.5: If we also train db_model together for the output of the model_rnn, it might work

In [43]:
def get_db_output(X,X_name,Y_true):
    pred = np.max(X,axis=-1)
    result = []
    for i in range(len(X)):
        p = pred[i]
        if p<0.5:
            result.append( np.zeros(vocab_len))
            continue
        v = np.argmax(X[i])
        if v == t.word_index[X_name[i]]:
            result.append(Y_true[i])
        else:
            if v in names:
                result.append(to_categorical(np.random.randint(len(actions)),num_classes=vocab_len))
            else:
                result.append(np.zeros(vocab_len))
                
    return np.array(result)

In [120]:
get_db_output(to_categorical([[t.word_index['being']]]),['john'],to_categorical([[t.word_index['being']]]))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [121]:
import keras.backend as K

In [122]:
def temp(m):
    wt=[]
    for w in m.weights:
        we=K.eval(w).flatten() 
        if we.shape==0: continue
        wt+=list(we)
    return np.array(wt)

In [123]:
dist = np.random.permutation(len(X_in))
batch_size=64
X_inr = X_in[dist]
X_name_r = np.array(X_name)[dist]
Y_inr = Y_in[dist]
for ii in range(100):
    batch_X = X_inr
    batch_Y = Y_inr
    batch_XN = X_name_r

    Xdb_pred = model_rnn.predict(batch_X)
    Ydb = get_db_output(Xdb_pred,batch_XN,batch_Y)
    db_model.fit(Xdb_pred,Ydb,batch_size=batch_size,epochs=1,validation_split=0.2)
    model.fit(batch_X,batch_Y,batch_size=batch_size,epochs=10,validation_split=0.2)
    if ii%5==0: db_model_fit(1)

Train on 1440 samples, validate on 360 samples
Epoch 1/1
1440/1440 [==============================] - 0s 65us/step - loss: 7.5227e-06 - val_loss: 7.2169e-06
Train on 1440 samples, validate on 360 samples
Epoch 1/10


KeyboardInterrupt: 

### Conclusion: The model sticks to one of the names like 'john', after which there is no gradient

### Hypothesis 3: If we penalize the model such that it produces output similar to the input it might give the correct answer

In [48]:
import keras.backend as K

In [49]:
import tensorflow as tf

In [79]:
l=0.2
def loss(y_true, y_pred):
    #y_pred[0] is the inner output, y_pred[1] is the outer output
    s = K.binary_crossentropy(y_true,y_pred)
    return s

def loss_inner(y_true,y_pred):
    a1 = K.batch_dot(y_true,y_pred,axes=[1,1])
    
    a2 = K.batch_dot(1-y_true,y_pred,axes=[1,1])
    pred=K.concatenate([a1,a2],axis=1)
    true=K.variable(value=[[1,0]])
    true=tf.tile(true,[tf.shape(y_true)[0],1])
    return l*K.categorical_crossentropy(true,pred)

In [80]:

out_3 = frozen_db_model(model_rnn.output)
model_3 = Model(inputs=[model_rnn.input],outputs=[model_rnn.output,out_3])

In [81]:
model_3.compile(optimizer='adam',loss=[loss_inner,loss])

In [82]:
dist = np.random.permutation(len(X_in))
batch_size=64
X_inr = X_in[dist]
X_name_r = np.array(X_name)[dist]
Y_inr = Y_in[dist]
for ii in range(10):
    batch_X = X_inr
    batch_Y = Y_inr
    batch_XN = X_name_r

    Xdb_pred = model_rnn.predict(batch_X)
    Ydb = get_db_output(Xdb_pred,batch_XN,batch_Y)
    db_model.fit(Xdb_pred,Ydb,batch_size=batch_size,epochs=1,validation_split=0.2)
    mid_in=np.array(np.sum(batch_X,axis=1)>0,dtype='int32')
    model_3.fit(batch_X,[mid_in,batch_Y],batch_size=batch_size,epochs=10,validation_split=0.2)
    if ii%5==0: db_model_fit(1)

Train on 1440 samples, validate on 360 samples
Epoch 1/1
1440/1440 [==============================] - 0s 66us/step - loss: 0.0366 - val_loss: 0.0363
Train on 1440 samples, validate on 360 samples
Epoch 1/10
1440/1440 [==============================] - 2s 1ms/step - loss: 0.2082 - dense_3_loss: 0.0667 - frozen_db_model_loss: 0.1415 - val_loss: 0.1857 - val_dense_3_loss: 0.0294 - val_frozen_db_model_loss: 0.1563
Epoch 2/10
1440/1440 [==============================] - 1s 390us/step - loss: 0.1801 - dense_3_loss: 0.0227 - frozen_db_model_loss: 0.1574 - val_loss: 0.1757 - val_dense_3_loss: 0.0187 - val_frozen_db_model_loss: 0.1570
Epoch 3/10
1440/1440 [==============================] - 1s 402us/step - loss: 0.1751 - dense_3_loss: 0.0165 - frozen_db_model_loss: 0.1586 - val_loss: 0.1731 - val_dense_3_loss: 0.0138 - val_frozen_db_model_loss: 0.1593
Epoch 4/10
1440/1440 [==============================] - 1s 401us/step - loss: 0.1732 - dense_3_loss: 0.0124 - frozen_db_model_loss: 0.1607 - val_l

Epoch 4/10
1440/1440 [==============================] - 1s 391us/step - loss: 0.0875 - dense_3_loss: 6.0932e-04 - frozen_db_model_loss: 0.0869 - val_loss: 0.0886 - val_dense_3_loss: 5.7557e-04 - val_frozen_db_model_loss: 0.0880
Epoch 5/10
1440/1440 [==============================] - 1s 408us/step - loss: 0.0875 - dense_3_loss: 5.8105e-04 - frozen_db_model_loss: 0.0869 - val_loss: 0.0886 - val_dense_3_loss: 5.5768e-04 - val_frozen_db_model_loss: 0.0880
Epoch 6/10
1440/1440 [==============================] - 1s 400us/step - loss: 0.0874 - dense_3_loss: 5.5282e-04 - frozen_db_model_loss: 0.0869 - val_loss: 0.0885 - val_dense_3_loss: 5.2471e-04 - val_frozen_db_model_loss: 0.0880
Epoch 7/10
1440/1440 [==============================] - 1s 393us/step - loss: 0.0874 - dense_3_loss: 5.2489e-04 - frozen_db_model_loss: 0.0869 - val_loss: 0.0885 - val_dense_3_loss: 4.9582e-04 - val_frozen_db_model_loss: 0.0880
Epoch 8/10
1440/1440 [==============================] - 1s 427us/step - loss: 0.0874 - d

KeyboardInterrupt: 

In [95]:
ii=1222
X[ii:ii+1]

['what is being done by amit?']

In [96]:
y=model_rnn.predict(X_in[ii:ii+1])
model_3.predict(X_in[ii:ii+1])

[array([[9.79744727e-05, 9.17079233e-05, 2.90108846e-06, 5.79227759e-08,
         6.49084342e-08, 1.03944608e-07, 1.63793146e-07, 3.55146085e-07,
         7.39774677e-08, 9.96569753e-01, 3.04787769e-04, 7.20099080e-04,
         7.03438476e-04, 8.67077091e-04, 6.39674894e-04, 1.36158604e-07,
         7.18421518e-08, 5.35556168e-08, 1.41160015e-07, 1.52004006e-07,
         7.61615908e-08, 1.19727844e-07, 1.61565637e-07, 1.10882326e-07,
         1.46888354e-07, 9.59928741e-08, 6.59995436e-08, 1.13530660e-07,
         8.32644815e-08, 8.02716187e-08, 2.36035376e-07]], dtype=float32),
 array([[8.25720326e-06, 1.41154806e-05, 1.28176061e-05, 1.37305924e-05,
         1.62413780e-05, 1.69007890e-05, 8.44069928e-06, 1.61130065e-05,
         1.02646682e-05, 7.36240463e-06, 1.21889325e-05, 1.32582418e-05,
         9.75085368e-06, 1.45800705e-05, 8.48550098e-06, 1.77180082e-01,
         1.28883228e-01, 1.39049562e-05, 1.81803882e-01, 1.16460496e-05,
         1.14678378e-05, 1.92572355e-01, 1.745348

In [97]:
y

array([[9.79744727e-05, 9.17079233e-05, 2.90108846e-06, 5.79227759e-08,
        6.49084342e-08, 1.03944608e-07, 1.63793146e-07, 3.55146085e-07,
        7.39774677e-08, 9.96569753e-01, 3.04787769e-04, 7.20099080e-04,
        7.03438476e-04, 8.67077091e-04, 6.39674894e-04, 1.36158604e-07,
        7.18421518e-08, 5.35556168e-08, 1.41160015e-07, 1.52004006e-07,
        7.61615908e-08, 1.19727844e-07, 1.61565637e-07, 1.10882326e-07,
        1.46888354e-07, 9.59928741e-08, 6.59995436e-08, 1.13530660e-07,
        8.32644815e-08, 8.02716187e-08, 2.36035376e-07]], dtype=float32)

In [98]:
t.index_word

{1: 'is',
 2: 'what',
 3: 'i',
 4: 'doing',
 5: 'being',
 6: 'done',
 7: 'by',
 8: 'upto',
 9: 'amit',
 10: 'ketan',
 11: 'aparna',
 12: 'dog',
 13: 'john',
 14: 'aman',
 15: 'singing',
 16: 'climbing',
 17: 'believe',
 18: 'riding',
 19: 'my',
 20: 'guess',
 21: 'dancing',
 22: 'skating',
 23: 'think',
 24: 'playing',
 25: 'guessdog',
 26: 'guessketan',
 27: 'guessamit',
 28: 'guessjohn',
 29: 'guessaman',
 30: 'guessaparna'}

In [99]:
db_model.predict(y)

array([[8.25720326e-06, 1.41154806e-05, 1.28176061e-05, 1.37305924e-05,
        1.62413780e-05, 1.69007890e-05, 8.44069928e-06, 1.61130065e-05,
        1.02646682e-05, 7.36240463e-06, 1.21889325e-05, 1.32582418e-05,
        9.75085368e-06, 1.45800705e-05, 8.48550098e-06, 1.77180082e-01,
        1.28883228e-01, 1.39049562e-05, 1.81803882e-01, 1.16460496e-05,
        1.14678378e-05, 1.92572355e-01, 1.74534872e-01, 1.21998401e-05,
        1.46989062e-01, 9.83875088e-06, 1.22070905e-05, 1.01957994e-05,
        1.14703207e-05, 1.10546935e-05, 1.17641175e-05]], dtype=float32)

In [100]:
[(t.index_word[i],y[0][i]) if i!=0 else '' for i in y.argsort(axis=-1)[0][-1::-1]]

[('amit', 0.99656975),
 ('john', 0.0008670771),
 ('aparna', 0.0007200991),
 ('dog', 0.0007034385),
 ('aman', 0.0006396749),
 ('ketan', 0.00030478777),
 '',
 ('is', 9.170792e-05),
 ('what', 2.9010885e-06),
 ('by', 3.5514608e-07),
 ('guessaparna', 2.3603538e-07),
 ('done', 1.6379315e-07),
 ('skating', 1.6156564e-07),
 ('my', 1.52004e-07),
 ('playing', 1.4688835e-07),
 ('riding', 1.4116002e-07),
 ('singing', 1.361586e-07),
 ('dancing', 1.1972784e-07),
 ('guessamit', 1.1353066e-07),
 ('think', 1.10882326e-07),
 ('being', 1.0394461e-07),
 ('guessdog', 9.5992874e-08),
 ('guessjohn', 8.326448e-08),
 ('guessaman', 8.027162e-08),
 ('guess', 7.616159e-08),
 ('upto', 7.397747e-08),
 ('climbing', 7.184215e-08),
 ('guessketan', 6.5999544e-08),
 ('doing', 6.4908434e-08),
 ('i', 5.7922776e-08),
 ('believe', 5.3555617e-08)]

In [101]:
Y_in[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [102]:
[to_categorical(t.word_index['skating'],num_classes=vocab_len)]

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32)]

In [103]:
db_model.predict(np.array([to_categorical(t.word_index['skating'],num_classes=vocab_len)]))

array([[6.3809131e-07, 6.3723871e-07, 1.3261200e-06, 1.1625814e-06,
        1.2117929e-06, 7.5841649e-07, 1.3209815e-06, 1.5397128e-06,
        7.1623776e-07, 8.6926002e-07, 1.2681979e-06, 1.3714529e-06,
        8.4603505e-07, 1.1133515e-06, 7.8123770e-07, 8.6682286e-07,
        8.9299738e-07, 1.7747351e-06, 1.2850279e-06, 9.9320982e-07,
        1.2195118e-06, 1.1679647e-06, 7.4824857e-07, 7.6966069e-07,
        1.0867199e-06, 1.2022308e-06, 1.0919276e-06, 9.5226477e-07,
        5.9434461e-07, 9.9766692e-07, 9.7066777e-07]], dtype=float32)

# CONCLUSION SUCCESS: The model converges. 
However, there is a problem

In [104]:
y=model_rnn.predict(to_categorical(pad_sequences(t.texts_to_sequences(["ketan"]),maxlen=10),num_classes=vocab_len))

In [105]:
y

array([[1.1782163e-03, 1.4886333e-03, 4.3853241e-05, 5.4093920e-07,
        6.0350908e-07, 8.2087166e-07, 7.8087805e-07, 9.1832135e-07,
        9.9023760e-07, 4.5584120e-02, 8.6722761e-01, 1.8921386e-02,
        7.7100899e-03, 1.8320702e-02, 3.9507180e-02, 7.2279369e-07,
        8.3128401e-07, 6.1744743e-07, 1.3881496e-06, 1.2875290e-06,
        8.4156670e-07, 9.6068334e-07, 1.2291653e-06, 6.0316040e-07,
        6.2408913e-07, 6.0711238e-07, 7.1035311e-07, 1.0104909e-06,
        5.1053144e-07, 1.0117416e-06, 6.1792809e-07]], dtype=float32)

In [106]:
np.argmax(y[0])

10

In [107]:
t.index_word[13]

'john'

# Generating full sentences

In [108]:
from keras.layers import RepeatVector

In [109]:
inp = Input((10,vocab_len))
encoder = LSTM(50,return_state=True)
out,out_h,out_c=encoder(inp)
out = Dense(vocab_len,activation='softmax')(out)
model_rnn_2=Model(inputs=[inp],outputs=out)

out_3 = frozen_db_model(model_rnn_2.output)
out_35 = RepeatVector(10)(out_3)
decoder = LSTM(50,return_sequences=True,return_state=True)
out_4,_,_= decoder(out_35,initial_state=[out_h,out_c])
out_4 = Dense(vocab_len,activation='softmax')(out_4)
model_4 = Model(inputs=[model_rnn_2.input],outputs=[model_rnn_2.output,out_4])

In [110]:
model_4.compile(optimizer='adam',loss=[loss_inner,keras.losses.categorical_crossentropy])

In [116]:
Y_in_full = [Y]

array([[[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       ...,

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.